In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

## 데이터 준비

In [2]:
origin=read_excel('행정동병합테스트22.xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,면수,하천및한강여부,아동인구,노인인구,장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,4592,0,612,2154,437,337.55,4,0.352898,0,32
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,12605,0,488,1808,267,60.79,0,0.063554,0,27
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,2237,0,108,626,110,956.51,3,1.000000,0,16
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,3533,유,385,1814,311,0.00,3,0.000000,10,34
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,9390,유,1033,3495,491,1.14,3,0.001192,10,54


In [3]:
origin.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수'],
      dtype='object')

In [4]:
origin.isna().sum()
origin1=origin.fillna(0)

## 결측치는 0으로 처리

> 실제 조사된 값이므로 없다면 관측되지 아니함으로 판단

In [5]:
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수'],
      dtype='object')

## 데이터 스케일링

In [6]:
scaler=MinMaxScaler()
df = origin1.copy()
df['의료기관minmax']=scaler.fit_transform(df[['의료기관_수']])
df['사회복지minmax']=scaler.fit_transform(df[['사회복지시설_수']])
df['보육시설minmax']=scaler.fit_transform(df[['보육시설수']])
df['노인인구minmax']=scaler.fit_transform(df[['노인인구']])
df['아동인구minmax']=scaler.fit_transform(df[['아동인구']])
df['장애인minmax']=scaler.fit_transform(df[['장애인_인구']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,4,0.352898,0,32,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,0.063554,0,27,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,3,1.000000,0,16,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,3,0.000000,10,34,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3,0.001192,10,54,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845


## 파생변수(`사회적약자_지원_인프라_밀집도`) 생성

In [7]:
df['사회적약자_지원_인프라_밀집도'] = (df['의료기관minmax'] + df['사회복지minmax'] + df['보육시설minmax']) / (df['노인인구minmax'] + df['아동인구minmax'] + df['장애인minmax'])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적minmax,수변점수,행정동별버스정류장수,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자지수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.352898,0,32,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.665309
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.063554,0,27,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,1.000000,0,16,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.809038
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.000000,10,34,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.660960
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.001192,10,54,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.734492


## 파생 변수의 값 정렬

In [8]:
df['사회적약자_지원_인프라_밀집도'].value_counts().sort_index()

사회적약자지수
0.011184     1
0.039567     1
0.050629     1
0.186262     1
0.192484     1
            ..
2.027138     1
3.103322     1
4.225994     1
10.164062    1
12.600828    1
Name: count, Length: 426, dtype: int64

## 결측치 확인

In [9]:
df['사회적약자_지원_인프라_밀집도'].isna().sum()

0

In [10]:
# df['사회적약자지수'].filln(0)
# df['사회적약자지수'].isnu().sum()

In [11]:
df['사회적약자_지원_인프라_밀집도'].unique()

array([6.65308983e-01, 4.36825837e-01, 8.09038155e-01, 6.60959535e-01,
       7.34491740e-01, 3.60259920e-01, 5.29775659e-01, 6.91660218e-01,
       1.95329781e+00, 1.75901854e+00, 1.18953654e+00, 4.32298961e-01,
       5.77930564e-01, 2.46883903e-01, 1.86261688e-01, 5.83676086e-01,
       4.12705017e-01, 3.10332222e+00, 1.44915896e+00, 4.22599380e+00,
       1.20231141e+00, 3.63840765e-01, 4.88222391e-01, 1.87502777e+00,
       3.16607992e-01, 5.21698121e-01, 4.61414511e-01, 3.22117284e-01,
       4.38746017e-01, 4.02971613e-01, 3.76665540e-01, 8.84842045e-01,
       7.80486935e-01, 2.39878714e-01, 1.36012945e+00, 4.96959102e-01,
       6.90418262e-01, 6.46659096e-01, 1.31682585e+00, 5.05748766e-01,
       5.06959597e-01, 3.60817258e-01, 3.96240964e-01, 9.59081076e-01,
       2.63136899e-01, 5.32704195e-01, 4.10137042e-01, 3.06717419e-01,
       5.02511285e-01, 8.94311423e-01, 7.43931161e-01, 5.90168330e-01,
       6.80997040e-01, 4.10746187e-01, 3.50202249e-01, 5.33505044e-01,
      

In [12]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수', '의료기관minmax', '사회복지minmax', '보육시설minmax', '노인인구minmax',
       '아동인구minmax', '장애인minmax', '사회적약자지수'],
      dtype='object')

## 지수 파일 생성에 사용될 컬럼 추출

In [14]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','의료기관_수','사회복지시설_수','보육시설수', '노인인구','아동인구','장애인_인구', '의료기관minmax', '사회복지minmax', '보육시설minmax', '노인인구minmax', '아동인구minmax', '장애인minmax','사회적약자_지원_인프라_밀집도'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,의료기관_수,사회복지시설_수,보육시설수,노인인구,아동인구,장애인_인구,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자지수
0,11110,종로구,1111051500,청운효자동,11,5,4,2154,612,437,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.665309
1,11110,종로구,1111053000,사직동,75,1,0,1808,488,267,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826
2,11110,종로구,1111054000,삼청동,11,0,3,626,108,110,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.809038
3,11110,종로구,1111055000,부암동,4,4,3,1814,385,311,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.660960
4,11110,종로구,1111056000,평창동,15,11,3,3495,1033,491,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.734492


## 지수 파일 생성

In [15]:
df1.to_excel('사회적약자_지원_인프라_밀집도1.xlsx', index=False)

In [16]:
df2=read_excel('사회적약자_지원_인프라_밀집도1.xlsx')
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,의료기관_수,사회복지시설_수,보육시설수,노인인구,아동인구,장애인_인구,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자지수
0,11110,종로구,1111051500,청운효자동,11,5,4,2154,612,437,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.665309
1,11110,종로구,1111053000,사직동,75,1,0,1808,488,267,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826
2,11110,종로구,1111054000,삼청동,11,0,3,626,108,110,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.809038
3,11110,종로구,1111055000,부암동,4,4,3,1814,385,311,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.660960
4,11110,종로구,1111056000,평창동,15,11,3,3495,1033,491,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.734492


## 이상치 `inf`, `-inf` 처리

In [17]:
# DataFrame(df2)에서 '사회적약자지수' 열을 선택
accessibility_series = df2['사회적약자_지원_인프라_밀집도']

# 'inf'와 '-inf'를 0으로 대체
accessibility_series = accessibility_series.replace([np.inf, -np.inf], 0)

# 변경된 '사회적약자지수' 열을 DataFrame에 할당
df2['사회적약자_지원_인프라_밀집도'] = accessibility_series

In [18]:
df2.to_excel("사회적약자_지원_인프라_밀집도1.xlsx", index=False)

In [19]:
df2 = read_excel("사회적약자_지원_인프라_밀집도1.xlsx")
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,의료기관_수,사회복지시설_수,보육시설수,노인인구,아동인구,장애인_인구,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자지수
0,11110,종로구,1111051500,청운효자동,11,5,4,2154,612,437,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.665309
1,11110,종로구,1111053000,사직동,75,1,0,1808,488,267,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826
2,11110,종로구,1111054000,삼청동,11,0,3,626,108,110,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.809038
3,11110,종로구,1111055000,부암동,4,4,3,1814,385,311,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.660960
4,11110,종로구,1111056000,평창동,15,11,3,3495,1033,491,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.734492


## 지수 정규화

In [20]:
df2['사회적약자_지원_인프라_밀집도minmax'] = scaler.fit_transform(df2[['사회적약자_지원_인프라_밀집도']])
df2

,시군구코드,시군구명,행정동코드,읍면동명,의료기관_수,사회복지시설_수,보육시설수,노인인구,아동인구,장애인_인구,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,사회적약자지수,사회적약자지수minmax
0,11110,종로구,1111051500,청운효자동,11,5,4,2154,612,437,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.665309,0.051957
1,11110,종로구,1111053000,사직동,75,1,0,1808,488,267,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.436826,0.033809
2,11110,종로구,1111054000,삼청동,11,0,3,626,108,110,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.809038,0.063374
3,11110,종로구,1111055000,부암동,4,4,3,1814,385,311,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.660960,0.051612
4,11110,종로구,1111056000,평창동,15,11,3,3495,1033,491,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.734492,0.057453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,62,6,9,4469,897,1075,0.096875,0.222222,0.230769,0.510127,0.137894,0.294021,0.583696,0.045475
422,11740,강동구,1174066000,성내제3동,80,6,11,4098,1024,913,0.125000,0.222222,0.282051,0.467674,0.157417,0.249589,0.719433,0.056257
423,11740,강동구,1174068500,길동,103,11,17,8478,2061,1988,0.160938,0.407407,0.435897,0.968875,0.316833,0.544432,0.548724,0.042697
424,11740,강동구,1174069000,둔촌제1동,1,0,0,11,1,3,0.001563,0.000000,0.000000,0.000000,0.000154,0.000000,10.164062,0.806447


## 최종 데이터 저장

In [21]:
df2.to_excel('사회적약자_지원_인프라_밀집도1.xlsx',index=False)